In [33]:
import tensorflow as tf
import pandas as pd
import sagemaker
tf.__version__

'2.1.3'

In [34]:
#Necessary Imports
import argparse, os
import boto3
import numpy as np
import pandas as pd
import sagemaker
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K

In [35]:
#Reading in data in folder
df = pd.read_csv('./data/Iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [36]:
#Splitting data in 80-20 split to use testing data for model inference later
train = df.iloc[:120,:]
test = df.iloc[121:,:]
train = train.drop('Id', axis=1)
test = test.drop('Id', axis=1)

In [37]:
#Train and test csv
train.to_csv('./data/train.csv', index=False)
test.to_csv('./data/test.csv', index=False)

In [38]:
#Create a sagemaker session to be able to upload data to s3
import boto3
sagemaker_session = sagemaker.Session()

In [39]:
#Uploading data to S3 bucket titled "tf-iris-data"
prefix = "tf-iris-data"
training_input_path = sagemaker_session.upload_data('./data/train.csv', key_prefix=prefix + '/training')

In [40]:
training_input_path

's3://sagemaker-us-east-1-678287862391/tf-iris-data/training/train.csv'

In [41]:
#ensure training data has uploaded properly
training_data = pd.read_csv(training_input_path, sep = ',')
training_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [42]:
#Need tf version and role for the Tensorflow Estimator
tf_version = tf.__version__
tf_version

'2.1.3'

In [43]:
#Sagemaker role, make sure you've allowed access to any S3 bucket
role = sagemaker.get_execution_role()
role

'arn:aws:iam::678287862391:role/service-role/AmazonSageMaker-ExecutionRole-20220413T144048'

In [44]:
# default S3 bucket
bucket = sagemaker_session.default_bucket()
# iterate through S3 objects and print contents
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
     print(obj.key)

tf-iris-data/training/train.csv


In [45]:
import configparser

config = configparser.ConfigParser()
config.read('env.ini')

# This is an object that represents the SageMaker session that we are currently operating in. This
# object contains some useful information that we will need to access later such as our region.
session = sagemaker.Session()

# SageMaker default SM_MODEL_DIR=/opt/ml/model
if os.getenv("SM_MODEL_DIR") is None:
    os.environ["SM_MODEL_DIR"] = config['SM']['SM_MODEL_DIR'] + '/model'

# SageMaker default SM_OUTPUT_DATA_DIR=/opt/ml/output
if os.getenv("SM_OUTPUT_DATA_DIR") is None:
    os.environ["SM_OUTPUT_DATA_DIR"] = config['SM']['SM_OUTPUT_DATA_DIR'] + '/output'

# SageMaker default SM_CHANNEL_TRAINING=/opt/ml/input/data/training
if os.getenv("SM_CHANNEL_TRAINING") is None:
    os.environ["SM_CHANNEL_TRAINING"] = config['SM']['SM_CHANNEL_TRAINING'] + '/data'

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    #Can have other hyper-params such as batch-size, which we are not defining in this case
    parser.add_argument('--epochs', type=int, default=40)
    parser.add_argument('--learning-rate', type=float, default=0.001)

    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAINING'))
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--output_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))

    args, _ = parser.parse_known_args()
    epochs     = args.epochs
    lr         = args.learning_rate
    model_dir  = args.model_dir
    training_dir   = args.train

In [48]:
#Use a tensorflow estimator from sagemaker to train model
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m4.xlarge',
                          framework_version=tf_version, 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 30
                          }
                         )

In [49]:
#Training
tf_estimator.fit({'train': training_input_path})

2022-04-13 10:11:27 Starting - Starting the training job...ProfilerReport-1649844687: InProgress
...
2022-04-13 10:12:16 Starting - Preparing the instances for training.........
2022-04-13 10:13:57 Downloading - Downloading input data...
2022-04-13 10:14:17 Training - Downloading the training image...
2022-04-13 10:14:57 Training - Training image download completed. Training in progress.2022-04-13 10:14:47,106 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-04-13 10:14:47,114 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-13 10:14:47,676 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-13 10:14:47,695 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-13 10:14:47,714 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-04-13 10:14:47,728 sagemaker-training-toolkit INFO     Inv

UnexpectedStatusException: Error for Training job tensorflow-training-2022-04-13-10-11-27-085: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/local/bin/python3.6 train.py --epochs 30 --model_dir s3://sagemaker-us-east-1-678287862391/tensorflow-training-2022-04-13-10-11-27-085/model"
Using TensorFlow backend.
Traceback (most recent call last):
  File "train.py", line 36, in <module>
    os.environ["SM_CHANNEL_TRAINING"] = config['SM']['SM_CHANNEL_TRAINING'] + '/data'
  File "/usr/local/lib/python3.6/configparser.py", line 959, in __getitem__
    raise KeyError(key)
KeyError: 'SM', exit code: 1